### Predicting Titanic Survival using..
#### **sklearn (Scikit Learn) machine learning (ML) library**
- **sklearn** is the most popular and robust library for machine learning, having a selection of algorithms that await your data

**titanic dataset** is an iconic dataset used for Machine Learning practice and training
- it contains 891 **observations** (rows) of 15 **variables** (columns) each
- the dataset is used to train ML models to predict the probability of surviving the Titanic disaster based on variables such as passenger class, age and gender (spoiler alert: first class female passengers had a *much* better chance of survival than third class males)
- **survived** column: 1 means survived; 0 means perished
- **sibsp** stands for siblings and spouses
- **parch** stands for parents and children
- some columns are repeats of each other:
  - **alive** (no, yes) is the string version of **survived** (0,1)
  - **pclass** (1,2,3) is the numeric version of **class** (First, Second, Third)
  - **who** (man, woman, child) overlaps **sex** (male, female)
  - **embarked** (S, C, Q) is short for **embark_town** (Southhampton, Cherbourg, Queenstown)

**seaborn** is another visualization / plotting library, similar to matplotlib
- seaborn also has the titanic dataset built in, so no csv file required
**Kaggle Titanic Competition**
- If you would like to submit your predictions to **Kaggle Titanic Competition**, use their datasets, which come pre-divided into:
    - train (**train-titanic.csv**) 
    - test (**test-titanic.csv**)

In [46]:
# import the usual basics:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import Image
import pprint as pp
import seaborn as sns

In [47]:
# import sklearn machine learning packages:

# LabelEncoder converts string data to numeric:
from sklearn.preprocessing import LabelEncoder

# normalize the data (get all values down to mean=0
# and all other values are standard deviation
# 95% of all values go in the +-2 std (-2 to 2)
from sklearn.preprocessing import StandardScaler

# import RandomForestClassifier for training model
from sklearn.ensemble import RandomForestClassifier

#### **Scikit Learn (sklearn) packages**
- **sklearn.preprocessing.LabelEncoder** encodes target labels with value between **0** and **n_classes-1**
  - all data must be numeric for ML model training
  - so, passenger classes First, Second and Third become 0, 1 and 2
  - we need to convert object (string) columns ("sex", "embarked") into numbers
- **sklearn.model_selection.train_test_split** splits data into training and testing sets
  - **model** is trained on training set (fed data with the answers)
  - **model** is tested on testing set by having it predict answers (testing labels)
- **sklearn.preprocessing.StandardScaler** is for getting all values in a standard range,  
where all mean values are 0; values within +/- 1 std are in the 1 to -1 range
  - model needs all values in same range, so that it doesn't
  conclude that age, which ranges from 0-80,  
  is 40 times more important in predicting survival than passenger class, which ranges from just 0-2

- make **confusion matrix** for each model (a 2x2 array showing TP, TN, FP, FN)
from sklearn.metrics import confusion_matrix

**What Is Random Forest?**
- random forest is a tree-based supervised learning algorithm
- **random forest classifier** can be used to solve for **regression** or **classification** problems
- Random forest tends to combine hundreds of decision trees and then trains each decision tree on a different sample of the observations.
- The final predictions of the random forest are made by averaging the predictions of each individual tree.

**What is Confusion Matrix?**
- a confusion maxtrix is a set of 4 values used to describe a categorical classification:
- a confusion maxtrix has 4 categories: True Positive, False Positive, True Negative, False Negative
  - **True Positive** : model said it's a 'cat' and it is a 'cat'
  - **True Negative** : model said it's not a 'cat' and that's right
  - **False Positive** : model said it's a 'cat' but it's not
  - **False Negative** : model said it's not a 'cat' but it is


In [48]:
# load titanic dataset from within seaborn (no csv file)
titanic_sns_df = sns.load_dataset('titanic')
print(titanic_sns_df.shape) # (891, 15)
titanic_sns_df.head()

(891, 15)


,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [49]:
titanic_sns_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.7+ KB


In [50]:
# load the same data again BUT this time from train_titanic.csv file provided by Kaggle
train_titanic_df = pd.read_csv('../csv/train_titanic.csv')
print(train_titanic_df.shape) # (891, 12)
train_titanic_df.iloc[508:612,:]

(891, 12)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
508,509,0,3,"Olsen, Mr. Henry Margido",male,28.0,0,0,C 4001,22.5250,NaN,S
509,510,1,3,"Lang, Mr. Fang",male,26.0,0,0,1601,56.4958,NaN,S
510,511,1,3,"Daly, Mr. Eugene Patrick",male,29.0,0,0,382651,7.7500,NaN,Q
511,512,0,3,"Webber, Mr. James",male,NaN,0,0,SOTON/OQ 3101316,8.0500,NaN,S
512,513,1,1,"McGough, Mr. James Robert",male,36.0,0,0,PC 17473,26.2875,E25,S
...,...,...,...,...,...,...,...,...,...,...,...,...
607,608,1,1,"Daniel, Mr. Robert Williams",male,27.0,0,0,113804,30.5000,NaN,S
608,609,1,2,"Laroche, Mrs. Joseph (Juliette Marie Louise La...",female,22.0,1,2,SC/Paris 2123,41.5792,NaN,C
609,610,1,1,"Shutes, Miss. Elizabeth W",female,40.0,0,0,PC 17582,153.4625,C125,S
610,611,0,3,"Andersson, Mrs. Anders Johan (Alfrida Konstant...",female,39.0,1,5,347082,31.2750,NaN,S


In [51]:
train_titanic_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [52]:
# load up the test set:
test_titanic_df = pd.read_csv('../csv/test_titanic.csv')

In [53]:
print(test_titanic_df.shape) # (418, 11)
test_titanic_df.tail()

(418, 11)


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S
417,1309,3,"Peter, Master. Michael J",male,NaN,1,1,2668,22.3583,NaN,C


In [54]:
# concat the two df's into one big df, w passenger ID's from 1-1309
# axis=0 so the concat happens along x-axis (horizontally), resulting
# in new rows, NOT new columns (for new cols specify axis=1)
titanic_df = pd.concat([train_titanic_df, test_titanic_df],axis=0)

In [55]:
print(titanic_df.shape) # (1309, 12)
# titanic_df[888:894]
# L@@K: The index starts over after 890 (due to concat)
# FIX: reset the index

(1309, 12)


In [56]:
titanic_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 0 to 417
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  1309 non-null   int64  
 1   Survived     891 non-null    float64
 2   Pclass       1309 non-null   int64  
 3   Name         1309 non-null   object 
 4   Sex          1309 non-null   object 
 5   Age          1046 non-null   float64
 6   SibSp        1309 non-null   int64  
 7   Parch        1309 non-null   int64  
 8   Ticket       1309 non-null   object 
 9   Fare         1308 non-null   float64
 10  Cabin        295 non-null    object 
 11  Embarked     1307 non-null   object 
dtypes: float64(3), int64(4), object(5)
memory usage: 132.9+ KB


In [57]:
# reset index, specify drop=True to prevent old wonky index from being preserved as a new column:
titanic_df.reset_index(drop=True,inplace=True)

In [58]:
titanic_df.sample(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
1172,1173,NaN,3,"Peacock, Master. Alfred Edward",male,0.75,1,1,SOTON/O.Q. 3101315,13.7750,NaN,S
446,447,1.0,2,"Mellinger, Miss. Madeleine Violet",female,13.00,0,1,250644,19.5000,NaN,S
48,49,0.0,3,"Samaan, Mr. Youssef",male,NaN,2,0,2662,21.6792,NaN,C
465,466,0.0,3,"Goncalves, Mr. Manuel Estanslas",male,38.00,0,0,SOTON/O.Q. 3101306,7.0500,NaN,S
1089,1090,NaN,2,"Baimbrigge, Mr. Charles Robert",male,23.00,0,0,C.A. 31030,10.5000,NaN,S
508,509,0.0,3,"Olsen, Mr. Henry Margido",male,28.00,0,0,C 4001,22.5250,NaN,S
918,919,NaN,3,"Daher, Mr. Shedid",male,22.50,0,0,2698,7.2250,NaN,C
1149,1150,NaN,2,"Bentham, Miss. Lilian W",female,19.00,0,0,28404,13.0000,NaN,S
747,748,1.0,2,"Sinkkonen, Miss. Anna",female,30.00,0,0,250648,13.0000,NaN,S
889,890,1.0,1,"Behr, Mr. Karl Howell",male,26.00,0,0,111369,30.0000,C148,C


In [59]:
# instantiate LabelEncoder
label_enc = LabelEncoder()

In [60]:
# Convert 'Sex' from 'female', 'male' to 0, 1
titanic_df['SexEnc'] = label_enc.fit_transform(titanic_df['Sex'])

In [61]:
print(titanic_df.shape) # (1309, 13)
titanic_df.head()

(1309, 13)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,SexEnc
0,1,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,1
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,0
2,3,1.0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,0
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,0
4,5,0.0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,1


In [62]:
# make a new column 'CabinKnown' with value of 1 Cabin exists (only 204 of those) and 0 if it is NA
# Use a vectorized check—no need for apply(lambda)
# .notna() returns a boolean
# .astype(int) casts the bool as int (0,1)
titanic_df['CabinKnown'] = titanic_df['Cabin'].notna().astype(int)

In [63]:
print(titanic_df.shape) # (1309, 14)
titanic_df.head()

(1309, 14)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,SexEnc,CabinKnown
0,1,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,1,0
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,0,1
2,3,1.0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,0,0
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,0,1
4,5,0.0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,1,0


In [64]:
# add the 'Fare' -- one of the 1309 is missing.. fill with median
median_fare = titanic_df['Fare'].median()
print(median_fare)

14.4542


In [65]:
titanic_df[ titanic_df['Fare'].isna() ] = median_fare

In [66]:
titanic_df['PassengerId'] = titanic_df['PassengerId'].astype(int)
titanic_df['Pclass'] = titanic_df['Pclass'].astype(int)
titanic_df['CabinKnown'] = titanic_df['CabinKnown'].astype(int)
titanic_df['SibSp'] = titanic_df['SibSp'].astype(int)
titanic_df['Parch'] = titanic_df['Parch'].astype(int)
titanic_df['SexEnc'] = titanic_df['SexEnc'].astype(int)

In [67]:
titanic_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  1309 non-null   int64  
 1   Survived     892 non-null    float64
 2   Pclass       1309 non-null   int64  
 3   Name         1309 non-null   object 
 4   Sex          1309 non-null   object 
 5   Age          1046 non-null   float64
 6   SibSp        1309 non-null   int64  
 7   Parch        1309 non-null   int64  
 8   Ticket       1309 non-null   object 
 9   Fare         1309 non-null   float64
 10  Cabin        296 non-null    object 
 11  Embarked     1307 non-null   object 
 12  SexEnc       1309 non-null   int64  
 13  CabinKnown   1309 non-null   int64  
dtypes: float64(3), int64(6), object(5)
memory usage: 143.3+ KB


In [68]:
# get the median Age for filling in the nearly 200 missing ages
median_age = titanic_df['Age'].median()
print(median_age)

28.0


In [69]:
# fill all missing ages with the median (28.0)
titanic_df['Age'].fillna(median_age,inplace=True)

In [70]:
titanic_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  1309 non-null   int64  
 1   Survived     892 non-null    float64
 2   Pclass       1309 non-null   int64  
 3   Name         1309 non-null   object 
 4   Sex          1309 non-null   object 
 5   Age          1309 non-null   float64
 6   SibSp        1309 non-null   int64  
 7   Parch        1309 non-null   int64  
 8   Ticket       1309 non-null   object 
 9   Fare         1309 non-null   float64
 10  Cabin        296 non-null    object 
 11  Embarked     1307 non-null   object 
 12  SexEnc       1309 non-null   int64  
 13  CabinKnown   1309 non-null   int64  
dtypes: float64(3), int64(6), object(5)
memory usage: 143.3+ KB


In [71]:
# fill 2 missing Embarked values with most common, 'S'
titanic_df[ titanic_df['Embarked'].isna() ] = 'S'

In [72]:
embarked_one_hot_enc_df = pd.get_dummies(titanic_df['Embarked'],dtype=int)

In [73]:
print(embarked_one_hot_enc_df.shape)
embarked_one_hot_enc_df.head()

(1309, 4)


,14.4542,C,Q,S
0,0,0,0,1
1,0,1,0,0
2,0,0,0,1
3,0,0,0,1
4,0,0,0,1


In [74]:
# merge emparked one hot encoded cols with main titanic df:
titanic_df = pd.concat([titanic_df,embarked_one_hot_enc_df],axis=1)

In [75]:
print(titanic_df.shape)
titanic_df.head()

(1309, 18)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,SexEnc,CabinKnown,14.4542,C,Q,S
0,1,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.25,NaN,S,1,0,0,0,0,1
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,0,1,0,1,0,0
2,3,1.0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.925,NaN,S,0,0,0,0,0,1
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1,C123,S,0,1,0,0,0,1
4,5,0.0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.05,NaN,S,1,0,0,0,0,1


In [76]:
# KISS: Make "Big X" simplest X feature / training set possible for Quik initial Kaggle submission
# , 'Age', 'Fare'
X = titanic_df[['Pclass', 'SexEnc', 'CabinKnown', 'S', 'C', 'Q']]
# [['Pclass', 'SexEnc', 'CabinKnown']] got a 0.77272 (good score)

In [83]:
cols = ['Pclass','SexEnc','CabinKnown']

# if there might be stray strings/empties:
titanic_df[cols] = (titanic_df[cols]
                    .replace({'': np.nan, 'nan': np.nan, 'NA': np.nan})
                    .apply(pd.to_numeric, errors='coerce'))   # -> floats if NaN appear
titanic_df[cols] = titanic_df[cols].astype('int64')  

IntCastingNaNError: Cannot convert non-finite values (NA or inf) to integer

In [81]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Pclass      1309 non-null   object
 1   SexEnc      1309 non-null   object
 2   CabinKnown  1309 non-null   object
 3   S           1309 non-null   int64 
 4   C           1309 non-null   int64 
 5   Q           1309 non-null   int64 
dtypes: int64(3), object(3)
memory usage: 61.5+ KB


In [82]:
X.head()

,Pclass,SexEnc,CabinKnown,S,C,Q
0,3,1,0,1,0,0
1,1,0,1,0,1,0
2,3,0,0,1,0,0
3,1,0,1,1,0,0
4,3,1,0,1,0,0


In [80]:
X['Pclass'] = X['Pclass'].astype(int)
X['SexEnc'] = X['SexEnc'].astype(int)
X['CabinKnown'] = X['CabinKnown'].astype(int)

ValueError: invalid literal for int() with base 10: 'S'

In [79]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Pclass      1309 non-null   object
 1   SexEnc      1309 non-null   object
 2   CabinKnown  1309 non-null   object
 3   S           1309 non-null   int64 
 4   C           1309 non-null   int64 
 5   Q           1309 non-null   int64 
dtypes: int64(3), object(3)
memory usage: 61.5+ KB


In [ ]:
X.info()

In [ ]:
X_train = X[:len(train_titanic_df)]

In [ ]:
print(X_train.shape)
X_train.tail()

(891, 6)


,Pclass,SexEnc,CabinKnown,S,C,Q
886,2,1,0,1,0,0
887,1,0,1,1,0,0
888,3,0,0,1,0,0
889,1,1,1,0,1,0
890,3,1,0,0,0,1


In [ ]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Pclass      891 non-null    object
 1   SexEnc      891 non-null    object
 2   CabinKnown  891 non-null    object
 3   S           891 non-null    int64 
 4   C           891 non-null    int64 
 5   Q           891 non-null    int64 
dtypes: int64(3), object(3)
memory usage: 41.9+ KB


In [ ]:
X_test = X[len(train_titanic_df):]

In [ ]:
print(X_test.shape)
X_test.head()

(418, 6)


,Pclass,SexEnc,CabinKnown,S,C,Q
891,3,1,0,0,0,1
892,3,0,0,1,0,0
893,2,1,0,0,0,1
894,3,1,0,1,0,0
895,3,0,0,1,0,0


In [ ]:
# save Survived as the target "little y_train"
# There is NO y_test (only Kaggle has those "answers")
y_train = train_titanic_df['Survived']

In [ ]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Pclass      891 non-null    object
 1   SexEnc      891 non-null    object
 2   CabinKnown  891 non-null    object
 3   S           891 non-null    int64 
 4   C           891 non-null    int64 
 5   Q           891 non-null    int64 
dtypes: int64(3), object(3)
memory usage: 41.9+ KB


In [ ]:
print(type(y_train[0]))

<class 'numpy.int64'>


In [ ]:
print(y_train.shape) # (891,)
y_train[:5]

(891,)


0    0
1    1
2    1
3    1
4    0
Name: Survived, dtype: int64

In [ ]:
# Now that we have X and y, we can train a model BUT first instantiate:
# instantiate RandomForestClassifier
rand_forest_model = RandomForestClassifier(
    n_estimators=10, criterion='entropy', random_state=42
)
# rand_forest_model = RandomForestClassifier(
#     n_estimators=400,       # many trees
#     max_depth=None,         # let trees grow, control with min_* instead
#     min_samples_split=4,
#     min_samples_leaf=1,
#     max_features="sqrt",    # good default for classification
#     bootstrap=True,
#     class_weight="balanced", # Titanic is imbalanced (~38% survived)
#     n_jobs=-1,
#     random_state=42,
#     criterion="gini"        # often edges out 'entropy' for RF
# )

In [ ]:
# train the RandomForestClassifier model on X_tran and y_train
rand_forest_model.fit(X_train,y_train)

ValueError: could not convert string to float: 'S'

In [ ]:
# have the model predict survival of the test set (X_test) 
# AGAIN : WE DO NOT HAVE y_test, those being the answers (ONLY Kaggle has that)
# Ergo: WE can only know how well our model did if we upload our predictions to Kaggle
y_pred = rand_forest_model.predict(X_test)

In [ ]:
print(type(y_pred[0]))

<class 'numpy.int64'>


In [ ]:
y_pred = y_pred.astype(int)

In [ ]:
print(type(y_pred[0]))

<class 'numpy.int64'>


In [ ]:
# check the model's predictions (even though we cannot know how accurate they are):
print(y_pred.shape) # (418,)
print(y_pred[:10])

(418,)
[0 0 1 1 0 0 0 0 1 0]


In [ ]:
# make the required Kaggle df: 
# MUST have exactly two cols: "PassengerId" and "Survived"
# "PassengerId" value: consec ints from 892-1309 (we can generate this w range())
# "Survived" value: our model's predictions as y_pred
kiss_titanic_pred_df = pd.DataFrame()

In [ ]:
kiss_titanic_pred_df["PassengerId"] = list(range(892,1310))
kiss_titanic_pred_df["Survived"] = y_pred

In [ ]:
print(kiss_titanic_pred_df.shape) # (418, 2)
kiss_titanic_pred_df.head()

(418, 2)


,PassengerId,Survived
0,892,0
1,893,0
2,894,1
3,895,1
4,896,0


In [ ]:
# save the predictions df to csv for uploading to Kaggle:
# specify index=False or else you get a new 'Unnamed: 0' col containing the index values -- which we definitely do NOT want as it will be rejected by Kaggle -- MUST have exactly TWO cols ONLYs
kiss_titanic_pred_df.to_csv('../csv/titanic_artmink_predictions.csv',index=False)

In [ ]:
# load the csv right back up to df to make sure it's good to go:
kiss_titanic_df = pd.read_csv('../csv/titanic_artmink_predictions.csv')

In [ ]:
print(kiss_titanic_df.shape) # (418, 2)
kiss_titanic_df.head()

(418, 2)


,PassengerId,Survived
0,892,0
1,893,0
2,894,1
3,895,1
4,896,0


In [ ]:
# predict 100% did not survive just to gain insight into the mysterious kaggle 'answer key'
no_survivors_df = pd.DataFrame()
no_survivors_df["PassengerId"] = list(range(892,1310))
no_survivors_df["Survived"] = 0
no_survivors_df.to_csv('../csv/titanic_no_survivors.csv',index=False)

In [ ]:
# split() passenger name to isolate the title:
passenger_names = [ "Boulos, Mr. Hanna", "Duff Gordon, Sir. Cosmo Edmund (Mr Morgan)",	
    "Jacobsohn, Mrs. Sidney Samuel (Amy Frances)", "Slabenoff, Mr. Petco", 
    "Olsen, Mr. Henry Margido", "Lang, Mr. Fang", "Daly, Mr. Eugene Patrick", 
    "Webber, Mr. James", "McGough, Mr. James Robert", 
    "Andersson, Mrs. Anders Johan (Alfrida Konstant)", "Jardin, Mr. Jose Neto", 
    "Laroche, Mrs. Joseph (Juliette Marie Louise)", "Shutes, Miss. Elizabeth W" ]

In [ ]:
# challenge: get the title as "Mr" from the name
pass_name = "Duff Gordon, Sir. Cosmo Edmund (Mr Morgan)"
# first, split the name on the comma:
pass_name_list = pass_name.split(",")
print(pass_name_list)
# get the 2nd of the 2 items:
pass_name_titled = pass_name_list[1]
print(pass_name_titled)
# split the titled name on the dot:
pass_title_name_list = pass_name_titled.split(".")
print(pass_title_name_list)
# get the 1st item from list--this is the title result we want:
# remove leading space
pass_title = pass_title_name_list[0].replace(" ","")
print(pass_title)

['Duff Gordon', ' Sir. Cosmo Edmund (Mr Morgan)']
 Sir. Cosmo Edmund (Mr Morgan)
[' Sir', ' Cosmo Edmund (Mr Morgan)']
Sir


In [ ]:
# process the list of passenger names on a loop that calls the extract_title_from_name() function
def extract_title(pass_name):
    # first, split the name on the comma:
    pass_name_list = pass_name.split(",")
    # print(pass_name_list)
    # get the 2nd of the 2 items:
    pass_name_titled = pass_name_list[1]
    # print(pass_name_titled)
    # split the titled name on the dot:
    pass_title_name_list = pass_name_titled.split(".")
    # print(pass_title_name_list)
    # get the 1st item from list--this is the title result we want: replace leading space with empty string
    pass_title = pass_title_name_list[0].replace(" ","")
    # print(pass_title)
    return pass_title

In [ ]:
# run a loop that iterates the passenger_names list, passing each name to function. Save return value of function -- the title -- to a new list:
passenger_titles = []
for pass_name in passenger_names:
    pass_title = extract_title(pass_name)
    passenger_titles.append(pass_title)

print(passenger_titles)

['Mr', 'Sir', 'Mrs', 'Mr', 'Mr', 'Mr', 'Mr', 'Mr', 'Mr', 'Mrs', 'Mr', 'Mrs', 'Miss']


In [ ]:
# make a new title column for the train_titanic.csv
train_titanic_df['Title'] = train_titanic_df['Name'].apply(lambda name : extract_title(name))

In [ ]:
train_titanic_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,Mr
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,Mrs
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,Miss
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,Mrs
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,Mr


In [ ]:
# make Title col again BUT chain all title-extractor code into inline lambda:
train_titanic_df['Title2'] = train_titanic_df['Name'].apply(lambda x : x.split(",")[1].split('.')[0].replace(" ",""))

In [ ]:
# print unique values from Title col
unique_titles = list(train_titanic_df["Title"].unique())
print(len(unique_titles))
print(unique_titles)

17
['Mr', 'Mrs', 'Miss', 'Master', 'Don', 'Rev', 'Dr', 'Mme', 'Ms', 'Major', 'Lady', 'Sir', 'Mlle', 'Col', 'Capt', 'theCountess', 'Jonkheer']


In [ ]:
train_titanic_df["Title"].value_counts()

Mr             517
Miss           182
Mrs            125
Master          40
Dr               7
Rev              6
Mlle             2
Major            2
Col              2
theCountess      1
Capt             1
Ms               1
Sir              1
Lady             1
Mme              1
Don              1
Jonkheer         1
Name: Title, dtype: int64

In [ ]:
# ['Mr', 'Mrs', 'Miss', 'Master', 'Don', 'Rev', 'Dr', 'Mme', 'Ms', 'Major', 'Lady', 'Sir', 'Mlle', 'Col', 'Capt', 'theCountess', 'Jonkheer']
# consolidate rare titles into two labels: "RareFemale" and "RareMale"
rare_male_titles = ['Don', 'Rev', 'Dr', 'Major', 'Sir', 'Col', 'Capt', 'Jonkheer']
rare_female_titles = ['Lady', 'theCountess', 'Ms', 'Mme', 'Mlle']
common_male_titles = ['Mr', 'Master']
common_female_titles = ['Mrs', 'Miss']
train_titanic_df['Title'] = train_titanic_df['Title'].apply(lambda x : 
    "RareFemale" if x in rare_female_titles 
    else "RareMale" if x in rare_male_titles 
    else "CommonFemale" if x in common_female_titles 
    else "CommonMale" if x in common_male_titles 
    else x)

In [ ]:
# re-print unique values from Title col
unique_titles = list(train_titanic_df["Title"].unique())
print(len(unique_titles))
print(unique_titles)

4
['CommonMale', 'CommonFemale', 'RareMale', 'RareFemale']


In [ ]:
train_titanic_df["Title"].value_counts()

CommonMale      557
CommonFemale    307
RareMale         21
RareFemale        6
Name: Title, dtype: int64

In [ ]:
# check survival of 'Master'
common_male_df = train_titanic_df[ train_titanic_df['Title']=='CommonMale' ]

In [ ]:
print(common_male_df.shape)

(557, 14)


In [ ]:
common_male_df['Survived'].value_counts()

0    453
1    104
Name: Survived, dtype: int64

In [ ]:
# check survival of 'RareFemale:
rare_female_df = train_titanic_df[ train_titanic_df['Title']=='RareFemale' ]

In [ ]:
rare_female_df['Survived'].value_counts()

1    6
Name: Survived, dtype: int64

In [ ]:
# check survival of 'RareMale' titles:
rare_male_df = train_titanic_df[ train_titanic_df['Title']=='RareMale' ]

In [ ]:
rare_male_df['Survived'].value_counts()

0    15
1     6
Name: Survived, dtype: int64

In [ ]:
# check survival of 'CommonMale' titles:
common_male_df = train_titanic_df[ train_titanic_df['Title']=='CommonMale' ]

In [ ]:
common_male_df['Survived'].value_counts()

0    453
1    104
Name: Survived, dtype: int64

In [ ]:
# check survival of 'CommonFemale' titles:
common_female_df = train_titanic_df[ train_titanic_df['Title']=='CommonFemale' ]

In [ ]:
common_female_df['Survived'].value_counts()

1    226
0     81
Name: Survived, dtype: int64

In [ ]:
# print column names for both for comparing:
print('seaborn columns:\n')
print('\nkaggle columns:\n')
# 3 L@@K: the seaborn data has a lot of redundant columns
# if we know gender and age, we can infer man/woman/child

seaborn columns:


kaggle columns:



In [ ]:
# print a few cols unique values to see overlapping and redundancy:
print()
print()

In [ ]:
# view statistical data for the numeric columns
# ti

In [ ]:
# tk_

In [ ]:
# get info about the df:
# tk_
# L@@K: we are missing a lot of ages and almost all Cabin values are null
# the move is to drop Cabin column altogether and drop missing Age rows
# BUT definitely DO KEEP Age column

In [ ]:
# drop the Cabin column:
# tk_

In [ ]:
print() # (891, 11)
# tk_

In [ ]:
# make a new new column for titanic kaggle called "who"
# set it equal to the "who" col from other titanic_df
# this way, we can know if the person is a child
# which is useful for filling missing ages w more accurate age
# tk_

In [ ]:
print()
# tk_

In [ ]:
# fill the 177 missing ages w mean age

In [ ]:
# calculate the mean age of the adults and the children
# man_
# man

In [ ]:
# mea
# me

In [ ]:
# mea
# mea

In [ ]:
# mea
# mea

- **df['col'].fillna(value)** is for filling with same value for all null / missing values
- if you need to fill based on a condition (man, woman, child), use **df['col'].apply(lambda)**

In [ ]:
# fill the 2 missing Embarked with the most common value, which is "S"
# add inplace=True if you don't want to save the changed df to itself
# tk_

In [ ]:
# tk_

In [ ]:
# fill the 177 missing ages with the correct mean age
# call apply lambda on the WHOLE df, since we need access to 2 different columns
# set the lamdba expression equal to row to capture the return value, row by row
# tk

#### **How the correct value is chosen from the dictionary passed to `map()`**

The correct value is chosen from the dictionary passed to `map()` based on the value in the `who` column for each row. Here's how it works:

1. **Mapping operation**: The `map()` function takes each value in the `who` column and looks it up in the dictionary `{'man': 33.2, 'woman': 32.0, 'child': 6.4}`.

2. **Dictionary lookup**: For each row, the `map()` function uses the value in the `who` column (either `'man'`, `'woman'`, or `'child'`) as the key to look up the corresponding value (either `33.2`, `32.0`, or `6.4`) in the dictionary.

3. **Assignment of values**: It then assigns that value (e.g., `33.2` for `'man'`) to the corresponding `Age` value, but only for rows where `Age` is `NaN` because we use `loc` to specifically target those rows.

This allows for a very efficient and direct mapping between the categories in the `who` column and the desired `Age` values.


In [ ]:
# shorter version: use loc to target only the rows where Age is NaN, and
# then set the values based on the who column. Here's a more concise solution:
# tk

In [ ]:
# tk

In [ ]:
# tk_

- **df['col'].value_counts()** returns an itemized count for each discrete category in the col

In [ ]:
# 'perished / survived' ratio of full 891 rows
# 549 / 342 (61.61% of the 891 did not survive)
# get the 'perished / survived' ratio (value_counts) for the 891 rows
# tk_ # 59.55% of 891 did not survive

In [ ]:
# get the breakdown of the 3 passenger classes
# tk_
# 'Pclass'

In [ ]:
# tk_

**groupby()**
- **group_count_df = df.groupby('col_1')[['col_2']].count()**
- **group_mean_df = df.groupby('col_1')[['col_2']].mean()**
- **group_mean_df** has index of 'col_1' where each category of the column is a row
- **group_mean_df** has one col, 'col_2', the values of which is the count (or mean) for that row category


In [ ]:
# get a break down of passengers by class:
# 'Pclass''Survived'
# p
# L@@K: careful -- the result is not how many survived per class,
# but how many passengers there were per class

In [ ]:
# pc

In [ ]:
# change the column with the value counts to 'count'
# pc

In [ ]:
# pc

In [ ]:
# do a groupby to get survival rates by sex
# 'Sex' 'Survived'
# su
print()
# su

In [ ]:
# do a groupby to get survival totals by sex
# co
# cou
print()
# cou

In [ ]:
# but this is not showing survival, but rather the number of males vs females
# the total is 891, which is the total number of observations (rows) in our df
# 'sex'

**double and triple groupby()**
- pass a list of columns to groupby() method

In [ ]:
# 'Sex' 'Pclass' 'Survived'
# 'Survived' 'Count'
# pc
# p
# pc

In [ ]:
# 'Sex' 'Pclass' 'Embarked' 'Survived'
# 'Survived' 'Count'
# pc
# pc
# pc

In [ ]:
# if 'survived' is actually showing total number of passengers
# how can we break down the triple group by survival?
# add a 4th item to group by list: 'survived'
# 'survived', 'sex', 'pclass', 'embarked'
# su
# L@@K: survived at left is 0 vs 1 -- the perished vs. survived breakdown we want to see

In [ ]:
# su

In [ ]:
# do a groupby to get survival totals by sex
# s
# sur
# surv

In [ ]:
# plot survived_counts_df by calling plot() on the survived_or_not_df dataframe
# set plot equal to ax so that we can show bar values
# ax

# 'Titanic Passengers: Survived vs. Perished')
# pl
# 'Number of Passengers'
# pl
# CHALLENGE: Label the bars with their counts: hint requires that ax variable as
# well as a for loop
# p
# plt'Perished','Survived'

# for bar in ax.containers:
#   ax.bar_label(bar, padding=5, color="coral")
# ax
print()
# p

**seaborn** provides another way to make bar charts, which are side-by-side numeric comparisons of a column's values:
- **sns.countplot(x='col_name', data=df)**
  - makes a bar chart comparing counts
  - does not require a "grouped df" (like survived_counts_df) for it to work
  - **x** is the 'col_name' ; category values appear below x-axis
  - **data** is the dataframe

In [ ]:
# make a seaborn countplot showing survived vs perished
# one bar per category of the 'survived' column;
# bar height is the count of that category
# ax

# 'Titanic Passengers: Survived vs. Perished')
# pl
# 'Perished','Survived'
# 'Number of Passengers'

# show values above bars
# ax

# pl

In [ ]:
# make a seaborn countplot counts of each passenger class
# ax

# 'Titanic Passengers by Class'
# p
# 'First','Second','Third'
# 'Number of Passengers'

# show values above bars
# for bar in ax.containers:
#   ax.bar_label(bar, padding=5, color="coral")
# a

# pl

**making a bar chart with data from two columns at once**
- **sns.countplot(x='col_1', hue='col_2', data=df)**
  - makes a bar chart comparing counts of *two* columns
  - **x** is the column 1; category values appear below x-axis
  - **hue** is column 2
  - **data** is the dataframe

In [ ]:
# make a seaborn countplot counts of each passenger class
# ax

# 'Titanic Survival by Class'
# pl
# 'Perished','Survived'
# 'Number of Passengers'
# plt
# show values above bars
# f
# ax.bar_label(ax.containers[0], padding=5, color="coral")

# pl

In [ ]:
# divide survived vs perished by sex
# 'Survived' 'Sex'
# ax

# 'Titanic Survival by Gender'
# pl
# 'Perished','Survived'
# 'Number of Passengers'
# pl
# show values above bars
# fo
# ax.bar_label(ax.containers[0], padding=5, color="coral")

# pl
# L@@K: More than half the passengers who embarked at Cherbourg survived
# vs. 2/3 of the passengers who got on at Southampton perished

**subdivide Pclass by Sex**
- create a new column that combines both Pclass and Sex
- then use this new column as the hue.

#### How to Arrange Bars in a Specific Order (1st Class Female/Male, 2nd Class Female/Male, 3rd Class Female/Male)

To arrange the bars in a specific order in your seaborn `countplot`, follow these steps:

#### Step 1: Set the desired order for the combined `Pclass_Sex` column
Convert the `Pclass_Sex` column into a categorical type and specify the order of categories:

```python
order = ['1_female', '1_male', '2_female', '2_male', '3_female', '3_male']
tk_df['Pclass_Sex'] = pd.Categorical(tk_df['Pclass_Sex'], categories=order, ordered=True)


In [ ]:
# make a new column that has as its value a concatenates string of Pclass and Sex
# '1_female', '1_male', '2_female', '2_male', '3_female', '3_male'
# 'Pclass_Sex'
# 'Pclass_Sex'

In [ ]:
# tk

In [ ]:
# survived vs perished by pclass AND sex using the new combo col as the hue
# 'Survived' 'Sex'
# 'Survived', 'Pclass_Sex'

# 'Titanic Survival by Class and Gender'
# pl
# 'Perished','Survived'
# 'Number of Passengers'
# pl
# show values above bars
# fo
# ax.bar_label(ax.containers[0], padding=5, color="coral")

# pl
# L@@K: More than half the passengers who embarked at Cherbourg survived
# vs. 2/3 of the passengers who got on at Southampton perished

In [ ]:
# Challenge" : plot survival by Embarked Port
# hint: no combo new column necessary, just an x for the main col
# and hue for the secondary column
# label the bars w their values
# divide survived vs perished by sex
# 'Survived' 'Sex'
# ax

# 'Titanic Survival by Port of Embarcation'
# pl
# 'Perished','Survived'
# 'Number of Passengers'
# pl
# show values above bars
# fo
  # ax

# rename the legend from S,Q,C to Southampton, Cherbourg, Queenstown
# make a dictonary with current legend labels as keys and new labels as values
# embarked_dict = {'S':'Southampton', 'Q':'Queenstown', 'C':'Cherbourg'}
# get handles and labels from the legend
# ha
# replace the labels

# use list comprehension to loop the dict, key by key:
# list comprehension provides a "one-liner" alternative to a loop:
# new_labels = [ embarked_dict[key] for key in labels ]
# new_labels = []
# for key, val in embarked_dict.items():
#   new_labels.append(val)

# or if you just know the new label names:

# update the legend
# 'Southampton', 'Queenstown', 'Cherbourg'
# "Port of Embarkation"

# pl
# L@@K: More than half the passengers who embarked at Cherbourg survived
# vs. 2/3 of the passengers who got on at Southampton perished

In [ ]:
# 1. Given a list of fruits, we want a new list of just the berries.
# Without list comprehension you would write a for loop containing an if statement
# that checked the current fruit to see if it contained the substring "berry":
# "apple", "banana", "blackberry", "blueberry", "cherry", "cranberry", "grape",
# "kiwi", "lemon", "mango", "peach", "raspberry", "strawberry", "tangerine", "watermelon"

# 2. Declare a list for holding the result (the berries)
# ber

# 3. Loop the list and check the condition, appending only berries to the new list
# for
print('for loop result:')

for loop result:



#### **map() vs filter**
- like **map(func,list)**, **filter(func,list)** takes 2 arguments:
- - a function and a list
- you can call a function or just run an inline anonymous lambda function
- each item is passes as the arg of the lambda function
- **map()** must return a new list w the SAME NUMBER of items as in the original
- **filter()** must return a new list w FEWER items than in the original
- the return value of filter is a **boolean comparison**
- only items that return **True** are accumulated in the the new list
- filter returns a filter object that you unpack by passing all to **list()**

**example: berries2 = list(filter(lambda fru : 'berry' in fru, fruits))**

In [ ]:
# ber
('berries2 via filter:')

'berries2 via filter:'

### list comprehension
- List comprehension offers a shorter way to make a new list from items in an existing list.
- The syntax is: **new_list = [return_value for_loop if_condition]**

In [ ]:
# list comprehension way:
# [ return_value for loop if condition ]
# ber
# 'berries3 via list comprehensio:'

**pivot tables: rows become columns, columns become rows**
- **df.pivot_table(index='col_1', columns='col_2', values='col_3')**
- index can have two or more levels by passing **index** a list
- **df.pivot_table(index=['col_1','col_2'], columns='col_3', values='col_4')**

In [ ]:
# make a pivot_table() where sex categories ('male', 'female') are the index values
# columns are passenger class string ('First', 'Second', 'Third') and values are 'survived'
# 'Sex', 'Pclass', 'Survived'
# "First","Second","Third"
# format as pcts: multiply the entire df by 100 -- this is a matrix operation, cuz it's 2D
# as opposed to a vector operation which is math done across all items in a column vector
# piv
# pi

In [ ]:
# Challenge: make a pivot table where:
# row names are passenger classes (First, Second, Third -- NOT 1,2,3)
# column names are sex
# values are survival pcts
# 'Survived'
# "First","Second","Third"
# pi
# piv

In [ ]:
# 'Embarked' 'Sex' 'Pclass' 'Survived'
# 'Sex','Embarked', 'Pclass', 'Survived'
# pivo
# p
# pivo

**label_encoder_fit_transform()** method takes a list of strings and returns corresponding numbers

In [ ]:
# test the label encoder on a simple dataset we make from scratch
# 'aardvark', 'barracuda', 'crocodile', 'dolphin', 'elephant',
#              'frog', 'giraffe', 'hawk', 'kangaroo', 'lizard', 'marlin',
#              'narhwahl', 'ostrich', 'pirana', 'quail', 'raccoon', 'salamander',

# 'mammal', 'fish', 'reptile', 'mammal', 'mammal', 'amphibian',
#               'mammal', 'bird', 'mammal', 'reptile', 'fish', 'mammal',
#               'bird', 'fish', 'bird', 'mammal', 'amphibian'


In [ ]:
# make dataframe from animals_dict:
# an
print() # (17, 2)
# a

**list(df['col_name'].unique()** returns a list of unique column values


In [ ]:
# an
# ani

**LabelEncoder.fit_transform(list_of_strings)** takes a list of strings and returns a corresponding list of numbers




In [ ]:
# instantiate a label encoder for converting string data to numeric:
# l

In [ ]:
# convert the animal classes (mammal, bird, fish, amphibian, reptile) to numbers 0-4
# 'class num'
# an

In [ ]:
# an

In [ ]:
# L@@K: we have 2 cols with string values that we need to make numeric
# sex has 2 categories (male, female), which need to be converted to 0,1
# embarked has 3 categories (S,C,Q), which needs to become 0,1,2

In [ ]:
# use label_encoder.fit_transform(list_of_string) to convert 'sex' and 'embarked' from strings to ints
# tk
# tk_

In [ ]:
# CHALLENGE: label encode 'Embarked' columns: change S C Q into 0,1,2
# tk
# tk_

In [ ]:
# SPLIT the DATA into TRAINING INPUTS and TRAINING LABELS
# X is the training inputs, which is all rows and all cols EXCEPT for the survived column
# the training inputs do not contain the survived column, because survived (0,1) is the answer
# that we want the model to be able to learn to predict
# X : titanic_kag_df[['']].va # X is all rows, all cols except 'survived'
# y .il : .va # y is all rows, just first col, so 'survived' col
# 'Pclass', 'Sex', 'Age'
# 'Pclass','Embarked','Sex','Age','Fare','SibSp','Parch'

In [ ]:
print()
# X_

In [ ]:
# 'Survived'
print() # (891,)
# y_

**StandardScaler.fit_transform(col)** creates an "even playing field" for the data
- it takes a column and recalculates all the values, such that the mean is 0  
- 1 standard deviation from the mean is assigned a 1 (or -1)
- works best for continuous values (age, fare) and NOT to be used on discrete values (pclass 1,2,3)
- for discrete values, we can use one hot encoding instead, whereby each class/category gets its own col (1st, 2nd, 3rd cols from pclass)
- bigger numbers (such as age compared to passenger class) are not judged to be more important
- 68% of the data will be in the -1 to 1 range (plus or minus 1 standard deviation from the mean)
- 95% of the data will be in the -2 to 2 range (plus or minus 1 standard deviation from the mean)



In [ ]:
# instantiate the StandardScaler object:
# sc

In [ ]:
# X

In [ ]:
# The y data is already in the form of 0 to 1 only, so
# we only need to scale that -- just scale the X (input) data
# use .loc to specify all rows as : and target cols as list of col names
# 'Age','Fare'

In [ ]:
# X_

In [ ]:
# 'images/random-forest-classifier-2.jpg'

In [ ]:
# 'images/random-forest-tree-w-text.jpg'

- **n_estimators=10**
- Definition: Specifies the number of trees in the forest (i.e., the number of decision trees).
- Explanation:
- In a Random Forest model, multiple decision trees are built, and their predictions are averaged (for regression) or voted on (for classification).
- The more trees you have, the more stable and accurate the model can be, although it may take longer to train.
- In this case: You are using 10 decision trees.
- **criterion='entropy'**
- Definition: This parameter specifies the function used to measure the quality of a split when constructing each tree.
- Explanation:
- The two common criteria are gini (Gini impurity) and entropy (information gain).
entropy: Measures how much information is gained by making a split. It uses the concept of information theory to find splits that reduce uncertainty (entropy) in the target labels.
gini: Measures the degree of "impurity" in the nodes and tends to be slightly faster.
- In this case: The Random Forest uses entropy to evaluate how splits reduce the uncertainty of the class labels in the dataset.
- **random_state=42**
- Definition: This parameter sets the seed for the random number generator.
Explanation:
- Random Forests introduce randomness by selecting random subsets of data for each tree and selecting random subsets of features for splitting at each node.
random_state ensures reproducibility by controlling this randomness. Using the same seed (e.g., 42) ensures the same results across different runs (all else being equal).

In [ ]:
# instantiate a RandomForestClassifier model:
# fo

**model.fit(X_train,y_train)** trains the model on the features with labels (that is, it gets the answers along with the questions)

In [ ]:
# fo

**predictions_list = model.predict(X_test)** returns a list of model predictions

In [ ]:
# load the test set
# tk

In [ ]:
print()
# t

In [ ]:
# tk_

In [ ]:
# there is no 'who' col of 'man', 'woman', 'child' to help fill missing ages
# so just fill all age NA w 30
# tk_t

In [ ]:
# tk_t

In [ ]:
# get the avg fare
# avg_
# avg_

In [ ]:
# fill that one and only missing Fare w mean:
# tk_

In [ ]:
# tk_

In [ ]:
# 'Pclass','Embarked','Sex','Age','Fare','SibSp','Parch'

In [ ]:
print()
# X_

In [ ]:
# Label Encode the Emparked and Sex cols -- strings become ints
# 'Embarked'

In [ ]:
# X_t

In [ ]:
# X_t

In [ ]:
# standard scaler the continuous values (non-categorical)
# 'Age','Fare'

In [ ]:
# X

**testing our model on the test.csv from kaggle**

In [ ]:
# have the new model predict survival on the kaggle 418 row test set
# sur

**imaginary passenger check**

In [ ]:
# imaginary passenger check
# the list must have all 7 expected "independent variables" in correct order
# Pclass	Sex	Age	SibSp	Parch	Fare	Embarked
# ['male' 'female'] = [1 0]
# ['S' 'C' 'Q'] = [2 0 1]

# inputs have to be a 2D matrix, so double wrap the list:
# 3	1	1	0.339424	-0.498407	0	0
# sec [2,	0, 20, 0,	0,	13.5,	0]
# thi [3,	1, 25, 0,	0,	9.5,	2]
# fir [1,	0, -0.3, 1,	0,	0.8,	1]
# [1,	0, -0.3, 1,	0,	0.8,	1],
#                                     [3,	1,  -0.3, 1,	0,	-0.5,	2],
#                                    [3,	1,  -0.3, 1,	0,	-0.5,	2],
#                                    [2,	1,  -0.3, 1,	0,	0.25,	1]
# ima

In [ ]:
# ima

In [ ]:
# imag

**Kaggle Titanic Survival Prediction Submission File Format:**.

- You should submit a csv file with exactly 418 entries plus a header row.
- Your submission will show an error if you have extra columns (beyond PassengerId and Survived) or rows.

- The file should have exactly 2 columns:

  - **PassengerId** (sorted in any order)
  - **Survived** (contains your binary predictions: 1 for survived, 0 for deceased)

In [ ]:
# make a df for our predictions to submit to kaggle
# kaggle requires a csv of 2 cols: PassengerId and Survived
# my_ti
# 'PassengerId'
# 'Survived'

In [ ]:
print()
# my

**saving a dataframe to csv**
- **df.to_csv(file_path/file_name, encoding='utf-8', index=False)**
- saves df as file_name to file_path
- encoding **encoding='utf-8'** just means normal English letters
- **index=False** means do not make a column for the index

In [ ]:
# save dataframe as csv
# index=False prevents the index, which are ints from 0-417 from becoming a 3rd col
# my_

In [ ]:
# load the csv to make sure it worked as expected
# my_
print() # (418, 3) oops -- we got an extra column
# my_
# kag_
# L@@K: we have an unwanted "Unnamed: 0" column of the old index values

- it's all good to go -- time to submit the csv file to kaggle
- download the csv file and upload it to kaggle at

 **https://www.kaggle.com/competitions/titanic**